In [32]:
import torch
from transformers import AutoTokenizer, AutoConfig

import numpy as np
import pandas as pd
import six
import shap

from TRC.models import ModelForWeightedSequenceClassification
from common_utils import extract_from_dataframe, mask_batch_seq_generator
from TRC.utils import evaluate, pad_sequences, simple_tokenize

from error_analysis_TRC import load_local_model, create_weight, prepare_data, tokenize_with_new_mask

In [33]:
ASSIGN_WEIGHT = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'roberta-large'

model_path = '/home/cc/rora_tesi_new/log/log_TRC/roberta-large/bertweet-seq/20_epoch/data/True_weight/42_seed/saved-model/pytorch_model.bin'
config_path = '/home/cc/rora_tesi_new/log/log_TRC/roberta-large/bertweet-seq/20_epoch/data/True_weight/42_seed/saved-model/config.json'

In [34]:
train_data_path = '/home/cc/rora_tesi_new/data/train.p'
test_data_path = '/home/cc/rora_tesi_new/data/test.p'

need_columns = ['tweet_tokens', 'sentence_class']

_, Y_train = prepare_data(train_data_path, need_columns)
X_test_raw, Y_test = prepare_data(test_data_path, need_columns)

test_batch_size = Y_test.shape[0]


In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_name, normalization = True)
model = load_local_model(model_path, config_path, device, model_name)
model = model.to(device)
labels = sorted(model.config.label2id, key=model.config.label2id.get)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
class_weight = None
if ASSIGN_WEIGHT:
    class_weight = create_weight(Y_train)

In [115]:
def prova(X):
    X_test, masks_test = tokenize_with_new_mask(X, 128, tokenizer)
    num_batches = X_test.shape[0] // len(X)
    print('aaaa',num_batches)
    
    test_batch_generator = mask_batch_seq_generator(X_test, Y_test, masks_test, 1)
    logits, y_true, test_loss, test_auc, test_acc, test_tn, test_fp, test_fn, test_tp, test_precision, test_recall, test_s_pred = evaluate(model,
                                                                                                test_batch_generator,
                                                                                                num_batches, device,
                                                                                                class_weight )

    return logits

In [117]:
prova(X_test_raw[0])

aaaa 1


100%|██████████| 1/1 [00:00<00:00, 27.74it/s]


tensor([[ 1.7821, -1.1009]])

In [124]:
explainer = shap.Explainer(prova, tokenizer, output_names=labels)

In [122]:
print(X_test_raw[0])

['@USER', 'As', 'much', 'fun', 'as', 'I', 'can', '.', 'Woke', 'up', 'with', 'food', 'poisoning', 'or', 'stomach', 'flu', '.', 'Been', 'bugging', 'me', 'all', 'day', '#tmi', 'Almost', 'done', 'driving', 'for', 'the', 'day']


In [128]:

shap_values = explainer(X_test_raw[0])


aaaa 1


100%|██████████| 1/1 [00:00<00:00, 29.12it/s]


aaaa 1


100%|██████████| 1/1 [00:00<00:00, 62.05it/s]


aaaa 1


100%|██████████| 1/1 [00:00<00:00, 61.93it/s]


AssertionError: The model produced 1 output rows when given 2 input rows! Check the implementation of the model you provided for errors.